In [1]:
import torch
from meu_dataset import MeuDataset,avaliar_descritor,calcular_matching
from teste_util import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_dataset = "./data/datasets/features_path_flowers_dataset.pt"
# Carregar o dataset do arquivo "meu_dataset.pt"
meu_dataset = MeuDataset.load_from_file(path_dataset)
#verificar se o objeto meu dataset está retornando o tensor correto
assert isinstance(meu_dataset,MeuDataset), 'o tipo de retorno não é MeuDataset'
assert isinstance(meu_dataset[0][0],torch.Tensor), 'o tipo de retorno não é torch.Tensor'


In [2]:
import gc
from torch.optim.lr_scheduler import ExponentialLR
gc.collect()
torch.cuda.empty_cache()
batch_size_siam = 30
from torch.utils.data import random_split, DataLoader


train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.4,0.4,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=batch_size_siam, shuffle=False)
dataloader_val = DataLoader(val_dataset, batch_size=batch_size_siam, shuffle=False)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size_siam, shuffle=False)

In [3]:
from torch.nn.functional import pairwise_distance
from tqdm import tqdm

# def loss_fn(output_pos, output_neg, margin=15,total=100): 
#     acertos_pos = torch.sum(output_pos[:, 0] == output_pos[:, 1])
#     false_pos = torch.sum(output_pos[:, 0] != output_pos[:, 1])
#     false_neg = output_neg.shape[0]
#     total_erros = false_pos + false_neg
    
#     loss = (total_erros-acertos_pos+margin).to(torch.float32).to(output_pos.device).detach().requires_grad_(True)
#     loss = torch.relu(loss)  # Aplicar ReLU para descartar os valores negativos
#     return loss/total

def loss_fn(output_pos, output_neg, margin=15,total=100): 
    acertos_pos = torch.sum(output_pos[:, 0] == output_pos[:, 1])
    false_pos = torch.sum(output_pos[:, 0] != output_pos[:, 1])
    false_neg = output_neg.shape[0]
    total_erros = false_pos + false_neg
    
    loss = (total_erros-acertos_pos+margin).to(torch.float32).to(output_pos.device).detach().requires_grad_(True)
    loss = torch.relu(loss)  # Aplicar ReLU para descartar os valores negativos
    # print("FN: ",false_neg,"FP:",false_pos, "VP: ",acertos_pos.item()," TF: ",total_erros.item()," L: ",loss.item(),"M :",margin)
    return loss/total

#Create methods to train the model
def train_one_epoch(model, data_loader, optimizer, loss_fn, device='cpu', is_training=True):
    model.train(is_training)
    total_loss = 0.
    # Definir os intervalos de colunas
    
    progress_bar = tqdm(data_loader)
    for idx,data in enumerate(progress_bar):
        #extrair as features e orientações
        batch_in,batch_out = data[0].to(device),data[1].to(device)        
        descs_original = model(batch_in)# encontrando descritores
        descs_transform = model(batch_out)# encontrando descritores
        match_pos,match_neg  = calcular_matching(descs_original, descs_transform,th=100.50)# calculando os matches
        loss = loss_fn(match_pos,match_neg ,margin=(batch_in.size(0)*3/2),total = batch_in.size(0)*4)
        
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_description(f'Loss: {loss.item()} - Total Loss: {total_loss}')
    return total_loss

In [17]:
from siamesa_e2cnn import Feature
n_channel =8
model =Feature(n_channel=n_channel).to(device)

/home/wagner/.local/lib/python3.11/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)


In [18]:
gc.collect()
torch.cuda.empty_cache()



def train(model,dataloader_train,dataloader_val):
    optimizer_siamese = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler = ExponentialLR(optimizer_siamese, gamma=0.75)
    best_loss = float('inf')
    best_model = None
    epochs_without_improvement = 0
    epochs = 50
    patience = 5
    
    for epoch in range(epochs):
        train_loss = train_one_epoch(model,dataloader_train , optimizer_siamese, loss_fn, device=device, is_training=True)
        with torch.no_grad():
            test_loss = train_one_epoch(model,dataloader_val , optimizer_siamese, loss_fn, device=device, is_training=False)

        # Verificar se a perda melhorou
        if test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
            best_model = model.state_dict()        
        else:
            epochs_without_improvement += 1
        
        if (epoch % 3 == 0) and (epoch != 0):
            scheduler.step()
            save_model(model, PATH_MODEL)
        
        # Verificar a condição de parada
        if epochs_without_improvement == patience:
            print(f"No improvement in loss for {epochs_without_improvement} epochs. Training stopped.")
            break

        print(f"Epoch [{epoch}/{epochs}] - Running Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Initial LR: {optimizer_siamese.param_groups[0]['initial_lr']:.6f}, Current LR: {optimizer_siamese.param_groups[0]['lr']:.6f}, Epochs without Improvement: {epochs_without_improvement}")

    # Carregar a melhor configuração do modelo
    model.load_state_dict(best_model)
    print(f'Epoch: {epoch}, Best Loss: {best_loss:.4f}')
    return model

PATH_MODEL = './data/models/feature_flowers_sp.pt'
model = train(model,dataloader_train,dataloader_val)

Loss: 0.375 - Total Loss: 26.741668671369553: 100%|██████████| 71/71 [00:05<00:00, 14.16it/s]              


Epoch [0/50] - Running Loss: 26.6917, Test Loss: 26.7417, Initial LR: 0.001000, Current LR: 0.001000, Epochs without Improvement: 0


Loss: 0.375 - Total Loss: 26.741668671369553: 100%|██████████| 71/71 [00:05<00:00, 14.09it/s]              


Epoch [1/50] - Running Loss: 26.6917, Test Loss: 26.7417, Initial LR: 0.001000, Current LR: 0.001000, Epochs without Improvement: 1


Loss: 0.375 - Total Loss: 26.741668671369553: 100%|██████████| 71/71 [00:05<00:00, 14.08it/s]              


...

Model saved to ./data/models/feature_flowers_sp.pt
Epoch [3/50] - Running Loss: 26.6917, Test Loss: 26.7417, Initial LR: 0.001000, Current LR: 0.000750, Epochs without Improvement: 3


Loss: 0.375 - Total Loss: 26.741668671369553: 100%|██████████| 71/71 [00:05<00:00, 13.95it/s]              


Epoch [4/50] - Running Loss: 26.6917, Test Loss: 26.7417, Initial LR: 0.001000, Current LR: 0.000750, Epochs without Improvement: 4


Loss: 0.375 - Total Loss: 26.741668671369553: 100%|██████████| 71/71 [00:05<00:00, 14.00it/s]              

No improvement in loss for 5 epochs. Training stopped.
Epoch: 5, Best Loss: 26.7417


In [19]:
save_model(model, PATH_MODEL)


Model saved to ./data/models/feature_flowers_sp.pt


In [22]:
PATH_MODEL = './data/models/feature_flowers_sp.pt'
model =Feature(n_channel=8)
model.to(device)
load_model(model, PATH_MODEL,device)

model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, model,th=0.4)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')

Model loaded from ./data/models/feature_flowers_sp.pt


100%|██████████| 36/36 [00:02<00:00, 14.34it/s]

Total de elementos no DataLoader: 4224
Acertei: 1969/2112 Errei: 90/2112


: 

In [8]:
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, model,th=0.32)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')

100%|██████████| 36/36 [00:00<00:00, 70.96it/s]

Total de elementos no DataLoader: 4224
Acertei: 1283/2112 Errei: 2/2112


### Refazer o treinamento para fazer o descritor na imagem original ao inves da feature

In [9]:
path_dataset = "./data/datasets/img_path_flowers_dataset.pt"
meu_dataset2 = MeuDataset.load_from_file(path_dataset)
train_dataset2, val_dataset2, test_dataset2 = random_split(meu_dataset2, [0.4,0.4,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train2 = DataLoader(train_dataset2, batch_size=batch_size_siam, shuffle=True)
dataloader_val2 = DataLoader(val_dataset2, batch_size=batch_size_siam, shuffle=True)
dataloader_test2 = DataLoader(test_dataset2, batch_size=batch_size_siam, shuffle=True)

In [10]:
n_channel =1
model =Feature(n_channel=n_channel).to(device)
PATH_MODEL = './data/models/img_flowers_sp.pt'
model = train(model,dataloader_train2,dataloader_val2)

Loss: 0.4166666865348816 - Total Loss: 39.48333543539047: 100%|██████████| 71/71 [00:01<00:00, 59.09it/s] 
Loss: 0.4583333432674408 - Total Loss: 39.291668742895126: 100%|██████████| 71/71 [00:01<00:00, 65.32it/s]


Epoch [0/50] - Running Loss: 39.4833, Test Loss: 39.2917, Initial LR: 100.001000, Current LR: 100.001000, Epochs without Improvement: 0


Loss: 0.4583333432674408 - Total Loss: 39.35833552479744: 100%|██████████| 71/71 [00:01<00:00, 64.86it/s]  
Loss: 0.4583333432674408 - Total Loss: 39.79166880249977: 100%|██████████| 71/71 [00:01<00:00, 70.68it/s] 


Epoch [1/50] - Running Loss: 39.3583, Test Loss: 39.7917, Initial LR: 100.001000, Current LR: 100.001000, Epochs without Improvement: 1


Loss: 0.4166666865348816 - Total Loss: 39.46666878461838: 100%|██████████| 71/71 [00:01<00:00, 63.03it/s] 
Loss: 0.375 - Total Loss: 39.00833550095558: 100%|██████████| 71/71 [00:01<00:00, 70.84it/s]              


...

Epoch [5/50] - Running Loss: 39.6000, Test Loss: 39.6167, Initial LR: 100.001000, Current LR: 75.000750, Epochs without Improvement: 3


Loss: 0.5 - Total Loss: 39.10000213980675: 100%|██████████| 71/71 [00:01<00:00, 66.93it/s]                 
Loss: 0.4583333432674408 - Total Loss: 39.12500211596489: 100%|██████████| 71/71 [00:00<00:00, 72.09it/s]  


Model saved to ./data/models/img_flowers_sp.pt
Epoch [6/50] - Running Loss: 39.1000, Test Loss: 39.1250, Initial LR: 100.001000, Current LR: 56.250563, Epochs without Improvement: 4


Loss: 0.4583333432674408 - Total Loss: 39.49166867136955: 100%|██████████| 71/71 [00:01<00:00, 64.91it/s] 
Loss: 0.5833333730697632 - Total Loss: 39.48333561420441: 100%|██████████| 71/71 [00:01<00:00, 68.62it/s] 

No improvement in loss for 5 epochs. Training stopped.
Epoch: 7, Best Loss: 39.0083


In [11]:
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test2, model,th=0.25)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')
save_model(model, PATH_MODEL)

100%|██████████| 36/36 [00:00<00:00, 73.73it/s]

Total de elementos no DataLoader: 4224
Acertei: 1163/2112 Errei: 7/2112
Model saved to ./data/models/img_flowers_sp.pt


In [12]:
from e2cnn import gspaces
from e2cnn import nn as enn    #the equivariant layer we need to build the model
from torch import nn
import torch

class Feature(nn.Module):
    def __init__(self,n_channel=2) -> None:
        super().__init__()
        r2_act = gspaces.Rot2dOnR2(N=12)

        feat_type_in  = enn.FieldType(r2_act,  n_channel*[r2_act.trivial_repr])
        feat_type_out = enn.FieldType(r2_act, n_channel*[r2_act.regular_repr])
        self.input_type = feat_type_in

        self.block1 = enn.SequentialModule(
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=3, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True),
                enn.GroupPooling(feat_type_out),
                )
        self.pool = enn.PointwiseAdaptiveAvgPool(self.block1.out_type,4)

    def forward(self,X1)->torch.Tensor:
        x = enn.GeometricTensor(X1, self.input_type)
        n_dim = X1.shape[-1]
        mask = enn.MaskModule(self.input_type, n_dim, margin=2).to(X1.device)
        x = mask(x)
        x = self.block1(x)
        x = self.pool(x)
        x = x.tensor.reshape(x.shape[0],-1)
        return x

n_channel =8
model =Feature(n_channel=n_channel).to(device)
image =torch.rand(4,n_channel,32,32).to(device)
model(image).shape

torch.Size([4, 128])